In [1]:
%pip install -q petals

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 732.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re
import json
import torch
from tqdm import tqdm
from transformers import BloomTokenizerFast 
from petals import DistributedBloomForCausalLM
from random import shuffle

In [ ]:
!git clone https://github.com/openai/grade-school-math.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## GSM8K. 

GSM8K состоит из 8,5 тыс. высококачественных школьных математических задач, созданных людьми, составляющими задачи. Они разделены на 7,5 тыс. обучающих задач и 1 тыс. тестовых задач. Для решения этих задач требуется от 2 до 8 шагов, и решения в основном включают выполнение последовательности элементарных вычислений с использованием основных арифметических операций (+ - / *) для получения окончательного ответа.

Файлы с тренировочными и тестовыми данными:
*   grade_school_math/data/train.jsonl
*   grade_school_math/data/test.jsonl

Каждая строка этих файлов соответствует одной математической задаче начальной школы, сохраненной в виде словаря json (с ключом «question» и ключом «answer»). Ответ отформатирован таким образом, что в нем используются аннотации вычислений, а окончательное числовое решение является последней строкой решения, которому предшествует ####

In [ ]:
train_path = './grade-school-math/grade_school_math/data/train.jsonl'
test_path = './grade-school-math/grade_school_math/data/test.jsonl'

Чтобы извлечь окончательное числовое решение для определенного вопроса, просто проанализируйте конец ответа, чтобы извлечь числовое значение сразу после токена ####.

Для удаления аннотаций калькулятора, необходимо удалить строки, начинающиеся с << и заканчивающиеся на >>.

In [ ]:
def make_prompt(cot_prompt, cot_labels):
    prompt = ''
    for example, label in zip(cot_prompt, cot_labels):
        prompt += 'Q: ' + example['question'] + '\nA: ' + example['answer'] + 'The answer is ' + str(label) + '\n\n'          
    return prompt

In [ ]:
def get_input_and_prompt(test_path, prompt_file_path = None):
    with open(test_path, 'r') as data_file:
        test_problems = data_file.readlines()
    # extract final answer
    answer_list = [float(re.search(r'#### ([0-9-]+)', problem).group(1)) for problem in test_problems]
    # remove annotations of calculator and final answer, convert string to dict
    input_list = [json.loads(re.sub('<<[0-9\(\)\.=+*/-]*>>|#### ([0-9-]+)', '', problem))
                  for problem in test_problems]
    combined = list(zip(answer_list, input_list))
    shuffle(combined)
    answer_list[:], input_list[:] = zip(*combined)
    if prompt_file_path is not None:
      with open(prompt_file_path, 'r') as prompt_file:
        prompt = prompt_file.read()
        cot_prompt = prompt
    else:
        cot_prompt = make_prompt(input_list[:11], answer_list[:11])
        answer_list = answer_list[11:]
        input_list = input_list[11:]
    return input_list, answer_list, cot_prompt

## BLOOM-176B

In [ ]:
MODEL_NAME = "bigscience/bloomz-petals"
tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
model = DistributedBloomForCausalLM.from_pretrained(MODEL_NAME, request_timeout=300, daemon_startup_timeout=120)
model = model.cuda()

## Chain-of-thoughts method with BLOOM-176B

Firstly, let's make chain-of-thougths prompts and input questions for bloom

In [ ]:
PROMPT_FILE_PATH = '/content/socratic_CoT_prompt'
RESULTS_FILE_PATH = '/content/drive/MyDrive/output_CoT_method'

In [ ]:
input_list, answer_list, cot_prompt = get_input_and_prompt(test_path, PROMPT_FILE_PATH)

In [ ]:
TASK_COUNT = len(input_list)
PARAMS = {
    "do_sample": None,
    "temperature": 1.0,
    "top_k": None,
    "top_p": None,
    "num_beams": 1,
    "max_new_tokens": 100,
    "num_return_sequences": None,
    "stop": ["Q:"]
}

In [ ]:
results = {
        "Parameters_of_generation": PARAMS,
        "Outputs": []
}
if RESULTS_FILE_PATH is not None:
    results = json.load(open(RESULTS_FILE_PATH, 'r'))
    TASK_COUNT = len(input_list)
    start_idx = len(results["Outputs"])
for input, answer in tqdm(zip(input_list[start_idx:TASK_COUNT], answer_list[start_idx:TASK_COUNT])):
    task_data = {
        "Question" : input['question'],
        "Answer": str(answer),
        "BLOOM_answer": None
    }
    
    tokenized_input = tokenizer(cot_prompt + 'Q: ' + input['question'] + '\nA: ', return_tensors="pt")["input_ids"].cuda()
    output = model.generate(tokenized_input, **PARAMS)
    shift = len(cot_prompt) + len(input['question']) + 3
    task_data["BLOOM_answer"] = tokenizer.decode(output[0])[shift:]
    results["Outputs"].append(task_data)
    with open(RESULTS_FILE_PATH, 'w') as output_file:
         json.dump(results, output_file)